In [1]:
import breeze.linalg._
import breeze.math._
import breeze.numerics._

In [14]:
print("**************************testing lambda")

**************************testing lambda

In [207]:
val A1  = DenseVector(3.0,3.0)
val A2  = DenseVector(4.0,4.0)
val A3  = DenseVector(5.0,5.0)

val B1  = DenseVector(17.0,17.0)
val B2  = DenseVector(19.0,19.0)

val C1  = DenseVector(21.0,21.0)
val C2  = DenseVector(99.0,99.0)
val C3  = DenseVector(13.0,13.0)
val C4  = DenseVector(11.0,11.0)

In [208]:
import org.apache.spark.rdd.RDD
val L1 = A1 :: A2 :: A3 :: List[DenseVector[Double]]()
val L2 = B1 :: B2 :: B3 :: List[DenseVector[Double]]()
val L3 = C1 :: C2 :: C3 :: List[DenseVector[Double]]()
val documentsRDD = sc.parallelize(List(A1,A2,A3,B1,B2,C1,C2,C3,C4), 4)
documentsRDD.collect() foreach println

In [221]:
//creat exact same data structure
//testing lambda

val metrics : RDD[( List[DenseVector[Double]] , Int )] =   
      documentsRDD.mapPartitions { docs => 
             val sigma_pt   =  DenseMatrix.zeros[Double](2,2) 
             val beta_pt    =  DenseVector.zeros[DenseMatrix[Double]](5)
             for(i <- 0 until beta_pt.size) { beta_pt(i) = DenseMatrix.zeros[Double](2,3) } 
             var bound_pt   = List[Double]()                         
             var lambda_pt  = List[DenseVector[Double]]()            
             
             var sz = 0
             docs.foreach { case (doc: DenseVector[Double]) => 
                                //lambda
                                sz = sz + 1
                                lambda_pt = doc :: lambda_pt
            }
            
            Iterator((lambda_pt, sz))
        
      }//end mapPartitions

In [216]:
metrics.collect() foreach println

(List(DenseVector(4.0, 4.0), DenseVector(3.0, 3.0)),2)
(List(DenseVector(17.0, 17.0), DenseVector(5.0, 5.0)),2)
(List(DenseVector(21.0, 21.0), DenseVector(19.0, 19.0)),2)
(List(DenseVector(11.0, 11.0), DenseVector(13.0, 13.0), DenseVector(99.0, 99.0)),3)


In [226]:
val collect_lambda: DenseMatrix[Double] = DenseMatrix.vertcat(metrics.map(_._1).flatMap(L=>L).collect().map(_.toDenseMatrix): _*)

In [231]:
print(collect_lambda)

4.0   4.0   
3.0   3.0   
17.0  17.0  
5.0   5.0   
21.0  21.0  
19.0  19.0  
11.0  11.0  
13.0  13.0  
99.0  99.0  

In [227]:
metrics.map(_._1).flatMap(l=>l).collect() foreach println

DenseVector(4.0, 4.0)
DenseVector(3.0, 3.0)
DenseVector(17.0, 17.0)
DenseVector(5.0, 5.0)
DenseVector(21.0, 21.0)
DenseVector(19.0, 19.0)
DenseVector(11.0, 11.0)
DenseVector(13.0, 13.0)
DenseVector(99.0, 99.0)


In [ ]:
print("**************************testing sigma")

In [126]:
//creat exact same data structure
//testing sigma

val metrics : RDD[( DenseMatrix[Double] , Int )] =   
      documentsRDD.mapPartitions { docs => 
             val sigma_pt   =  DenseMatrix.zeros[Double](2,2) 
             val beta_pt    =  DenseVector.zeros[DenseMatrix[Double]](5)
             for(i <- 0 until beta_pt.size) { beta_pt(i) = DenseMatrix.zeros[Double](2,3) } 
             var bound_pt   = List[Double]()                         
             var lambda_pt  = List[DenseVector[Double]]()            
             
             var sz = 0
             docs.foreach { case (doc: DenseVector[Double]) => 
                                //lambda
                                sz = sz + 1
                                sigma_pt += DenseMatrix((25.0,25.0),(25.0,25.0))

            }
            
            Iterator((sigma_pt, sz))
        
      }//end mapPartitions

In [127]:
metrics.collect() foreach println

(50.0  50.0  
50.0  50.0  ,2)
(50.0  50.0  
50.0  50.0  ,2)
(50.0  50.0  
50.0  50.0  ,2)
(75.0  75.0  
75.0  75.0  ,3)


In [128]:
val collect_sigma : DenseMatrix[Double] = metrics.map(_._1).treeAggregate(DenseMatrix.zeros[Double](2,2))(_ += _, _ += _)

In [129]:
print(collect_sigma)

225.0  225.0  
225.0  225.0  

In [175]:
print("**************************testing beta")

val A1  = DenseMatrix((2.0,2.0),(2.0,2.0))
val A2  = DenseMatrix((4.0,4.0),(4.0,4.0))
val A3  = DenseMatrix((6.0,6.0),(6.0,6.0))
val A4  = DenseMatrix((5.0,5.0),(5.0,5.0))
val A5  = DenseMatrix((25.0,25.0),(25.0,25.0))

**************************testing beta

In [202]:
//creat exact same data structure
//testing beta

val metrics : RDD[( DenseVector[DenseMatrix[Double]] , Int )] =   
      documentsRDD.mapPartitions { docs => 
             val sigma_pt   =  DenseMatrix.zeros[Double](2,2) 
             val beta_pt    =  DenseVector.zeros[DenseMatrix[Double]](5)
             val dummy      = DenseVector(A1,A2,A3,A4,A5)
             for(i <- 0 until beta_pt.size) { beta_pt(i) = DenseMatrix.zeros[Double](2,2) } 
             var bound_pt   = List[Double]()                         
             var lambda_pt  = List[DenseVector[Double]]()            
             
             var sz = 0
             docs.foreach { case (doc: DenseVector[Double]) => 
                                //lambda
                                sz = sz + 1
                                beta_pt(sz)(::, List(0)) += Matrix((25.0),(25.0))
             }
            
            Iterator((beta_pt, sz))
        
      }//end mapPartitions

In [203]:
metrics.collect() foreach println

(DenseVector(0.0  0.0  
0.0  0.0  , 25.0  0.0  
25.0  0.0  , 25.0  0.0  
25.0  0.0  , 0.0  0.0  
0.0  0.0  , 0.0  0.0  
0.0  0.0  ),2)
(DenseVector(0.0  0.0  
0.0  0.0  , 25.0  0.0  
25.0  0.0  , 25.0  0.0  
25.0  0.0  , 0.0  0.0  
0.0  0.0  , 0.0  0.0  
0.0  0.0  ),2)
(DenseVector(0.0  0.0  
0.0  0.0  , 25.0  0.0  
25.0  0.0  , 25.0  0.0  
25.0  0.0  , 0.0  0.0  
0.0  0.0  , 0.0  0.0  
0.0  0.0  ),2)
(DenseVector(0.0  0.0  
0.0  0.0  , 25.0  0.0  
25.0  0.0  , 25.0  0.0  
25.0  0.0  , 25.0  0.0  
25.0  0.0  , 0.0  0.0  
0.0  0.0  ),3)


In [181]:
val collect_beta  : DenseVector[DenseMatrix[Double]] = metrics.map(_._1).reduce((vec1,vec2) => vec1 :+ vec2)

In [184]:
collect_beta.map(x => {println(x); print("\n")} )

DenseVector((), (), (), (), ())

0.0  0.0  
0.0  0.0  

100.0  100.0  
100.0  100.0  

100.0  100.0  
100.0  100.0  

25.0  25.0  
25.0  25.0  

0.0  0.0  
0.0  0.0  



In [ ]:
print("**************************testing bound")

In [111]:
//creat exact same data structure
//testing bond

val metrics : RDD[( List[Double] , Double )] =   
      documentsRDD.mapPartitions { docs => 
             val sigma_pt   =  DenseMatrix.zeros[Double](2,2) 
             val beta_pt    =  DenseVector.zeros[DenseMatrix[Double]](5)
             for(i <- 0 until beta_pt.size) { beta_pt(i) = DenseMatrix.zeros[Double](2,3) } 
             var bound_pt   = List[Double]()                         
             var lambda_pt  = List[DenseVector[Double]]()            
             
             var sz = 0.0
             docs.foreach { case (doc: DenseVector[Double]) => 
                                //lambda
                                sz = sz + 1.0
                                bound_pt = sz :: bound_pt
            }
            
            Iterator((bound_pt, sz))
        
      }//end mapPartitions

In [112]:
metrics.collect() foreach println

(List(2.0, 1.0),2.0)
(List(2.0, 1.0),2.0)
(List(2.0, 1.0),2.0)
(List(3.0, 2.0, 1.0),3.0)


In [113]:
val collect_bound : DenseVector[Double] = DenseVector(metrics.map(_._1).reduce{(prev, item) => prev ++ item}.toArray) 

In [114]:
print(collect_bound)

DenseVector(2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 3.0, 2.0, 1.0)